## Questão 03 - Riyadh Levi

In [1]:
# Importando pacote request externo ao Python
import requests

In [2]:
# Definindo uma função para efetuar o download da página
def download(url, num_retries=2):
    print('Downloading: ', url)
    page = None
    try:
        response = requests.get(url)
        page = response.text
        if response.status_code >= 400:
            print('Download error:', response.text)
            if num_retries and 500 <= response.status_code < 600:
                return download(url, num_retries - 1)
    except requests.exceptions.RequestExceptions as e:
        print('Download error: ', e.reason)
    return page

In [14]:
# Importanto pacote externo ao Python, efetuando o download do site e iniciando a 'sopa'
from bs4 import BeautifulSoup
url = 'https://www.climatempo.com.br/previsao-do-tempo/cidade/264/teresina-pi'
html = download(url)
soup = BeautifulSoup(html, 'html.parser')

Downloading:  https://www.climatempo.com.br/previsao-do-tempo/cidade/264/teresina-pi


In [87]:
tempMax = soup.find(id='tempMax0').get_text()

In [88]:
tempMin = soup.find(id='tempMin0').get_text()

In [42]:
demaisDados = soup.find_all(class_='left text-center small-12 top5 normal font12 txt-black')

In [72]:
dadosLimpos = []
for dado in demaisDados:
    dado_limpo = dado.get_text().replace('\n', '').replace('            ','')
    dadosLimpos.append(dado_limpo)
    

In [73]:
dadosLimpos

['10mm90%', '83%43%', 'ENE6km/h', '05:25h17:46h']

In [132]:
infoChuva = dadosLimpos[0].replace('%','').split('mm')
infoAr    = dadosLimpos[1].split('%')
infoVento = dadosLimpos[2].replace('km/h','').split('ENE')
infoSol   = dadosLimpos[3].replace(':', '').split('h')
infoAr.remove(infoAr[2])
infoVento.remove(infoVento[0])
infoSol.remove(infoSol[2])

In [133]:
print(infoChuva, infoAr, infoVento, infoSol)

['10', '90'] ['83', '43'] ['6'] ['0525', '1746']


In [83]:
# Importando pacote externo ao Python
import sqlite3

In [84]:
# Criando conexão com o banco
con = sqlite3.connect('questao_03.db')

In [86]:
# Criando uma tabela
con.execute(''' CREATE TABLE PREVISAO
            (ID INT PRIMARY KEY NOT NULL,
            TEMPMAX INT NOT NULL,
            TEMPMIN INT NOT NULL,
            CHUVAMM INT NOT NULL,
            CHUVAPOR INT NOT NULL,
            UMIDMAX INT NOT NULL,
            UMIDMIN INT NOT NULL,
            VENTOVEL INT NOT NULL,
            SOLNASCE CHAR(10),
            SOLSEPOE CHAR(10));''')
print ('Table created successfully!')

con.close()

Table created successfully!


### Insert Operation

In [134]:
con = sqlite3.connect('questao_03.db')
print('Iniciando conexão com o banco...')
con.execute("INSERT INTO PREVISAO (ID,TEMPMAX,TEMPMIN,CHUVAMM, CHUVAPOR, UMIDMAX, UMIDMIN, VENTOVEL, SOLNASCE, SOLSEPOE) \
    VALUES(2, %s, %s, %s, %s, %s, %s, %s, %s, %s);"%(tempMax.replace('°', ''), tempMin.replace('°',''), infoChuva[0], infoChuva[1],infoAr[0],infoAr[1],infoVento[0],infoSol[0],infoSol[1]))
con.commit()
print('Gravação realizada com sucesso!')
con.close()
print('Finalizando conexão com o banco')

Iniciando conexão com o banco...
Gravação realizada com sucesso!
Finalizando conexão com o banco


### Select Operation

In [135]:
con = sqlite3.connect('questao_03.db')
print('Iniciando conexeção com o banco...')
cursor = con.execute('SELECT id, tempmax, tempmin, chuvamm, chuvapor, umidmax, umidmin, ventovel, solnasce, solsepoe from PREVISAO')
for row in cursor:
    print('ID = ', row[0])
    print('TEMPMAX = ', row[1])
    print('TEMPMIN = ', row[2])
    print('CHUVAMM = ', row[3])
    print('CHUVAPOR = ', row[4])
    print('UMIDMAX = ', row[5])
    print('UMIDMIN = ', row[6])
    print('VENTOVEL = ', row[7])
    print('SOLNASCE = ', row[8])
    print('SOLSEPOE = ', row[9],'\n')

print('Operação realizada com sucesso!')
con.close()
print('Finalizando conexão com o banco')

Iniciando conexeção com o banco...
ID =  1
TEMPMAX =  36
TEMPMIN =  24
CHUVAMM =  10
CHUVAPOR =  90
UMIDMAX =  83
UMIDMIN =  43
VENTOVEL =  6
SOLNASCE =  -20
SOLSEPOE =  -29 

ID =  2
TEMPMAX =  36
TEMPMIN =  24
CHUVAMM =  10
CHUVAPOR =  90
UMIDMAX =  83
UMIDMIN =  43
VENTOVEL =  6
SOLNASCE =  525
SOLSEPOE =  1746 

Operação realizada com sucesso!
Finalizando conexão com o banco


### Unindo todas as partes temos...

In [7]:
# Importando pacote externo ao Python
import sqlite3
# Importando pacote request externo ao Python
import requests
# Importanto pacote externo ao Python, e
from bs4 import BeautifulSoup
# Importando time para controlar a frequência de requisições
import time

# Definindo uma função para efetuar o download da página
def download(url, num_retries=2):
    print('Downloading: ', url)
    page = None
    try:
        response = requests.get(url)
        page = response.text
        if response.status_code >= 400:
            print('Download error:', response.text)
            if num_retries and 500 <= response.status_code < 600:
                return download(url, num_retries - 1)
    except requests.exceptions.RequestExceptions as e:
        print('Download error: ', e.reason)
    return page

# Criando um contador para delimitar o loop
count = 0;
# Loop para execução do scrape
while count < 5:
    # Efetuando o download do site e iniciando a 'sopa'
    url = 'https://www.climatempo.com.br/previsao-do-tempo/cidade/264/teresina-pi'
    html = download(url)
    soup = BeautifulSoup(html, 'html.parser')
    
    # Criação das variáveis, primeiramente elas recebem 'valores crus'
    tempMax = soup.find(id='tempMax0').get_text()
    tempMin = soup.find(id='tempMin0').get_text()
    demaisDados = soup.find_all(class_='left text-center small-12 top5 normal font12 txt-black')
    
    # Tratamento / Limpeza dos dados
    dadosLimpos = []
    for dado in demaisDados:
        dado_limpo = dado.get_text().replace('\n', '').replace('            ','')
        dadosLimpos.append(dado_limpo)

    infoChuva = dadosLimpos[0].replace('%','').split('mm')
    infoAr    = dadosLimpos[1].split('%')
    if dadosLimpos[2].count('ENE') == 1:
        infoVento = dadosLimpos[2].replace('km/h','').split('ENE')
    elif dadosLimpos[2].count('ESE') == 1:
        infoVento = dadosLimpos[2].replace('km/h','').split('ESE')
    infoSol   = dadosLimpos[3].replace(':', '').split('h')
    infoAr.remove(infoAr[2])
    infoVento.remove(infoVento[0])
    infoSol.remove(infoSol[2])
    
    # Insert - Banco de Dados
    con = sqlite3.connect('questao_03.db')
    print('\nIniciando conexão com o banco...')
    con.execute("INSERT INTO PREVISAO (ID,TEMPMAX,TEMPMIN,CHUVAMM, CHUVAPOR, UMIDMAX, UMIDMIN, VENTOVEL, SOLNASCE, SOLSEPOE) \
    VALUES(%d, %s, %s, %s, %s, %s, %s, %s, %s, %s);"%(count+11, tempMax.replace('°', ''), tempMin.replace('°',''), infoChuva[0], infoChuva[1],infoAr[0],infoAr[1],infoVento[0],infoSol[0],infoSol[1]))
    con.commit()
    print('Gravação realizada com sucesso!')
    con.close()
    print('Finalizando conexão com o banco')

    # Testando 
    con = sqlite3.connect('questao_03.db')
    print('\nIniciando conexeção com o banco...')
    cursor = con.execute('SELECT id, tempmax, tempmin, chuvamm, chuvapor, umidmax, umidmin, ventovel, solnasce, solsepoe from PREVISAO')
    for row in cursor:
        print('ID = ', row[0])
        print('TEMPMAX = ', row[1])
        print('TEMPMIN = ', row[2])
        print('CHUVAMM = ', row[3])
        print('CHUVAPOR = ', row[4])
        print('UMIDMAX = ', row[5])
        print('UMIDMIN = ', row[6])
        print('VENTOVEL = ', row[7])
        print('SOLNASCE = ', row[8])
        print('SOLSEPOE = ', row[9],'\n')

    print('Operação realizada com sucesso!')
    con.close()
    print('Finalizando conexão com o banco')
    time.sleep(20)
    count+=1




Downloading:  https://www.climatempo.com.br/previsao-do-tempo/cidade/264/teresina-pi

Iniciando conexão com o banco...
Gravação realizada com sucesso!
Finalizando conexão com o banco

Iniciando conexeção com o banco...
ID =  1
TEMPMAX =  36
TEMPMIN =  24
CHUVAMM =  10
CHUVAPOR =  90
UMIDMAX =  83
UMIDMIN =  43
VENTOVEL =  6
SOLNASCE =  -20
SOLSEPOE =  -29 

ID =  2
TEMPMAX =  36
TEMPMIN =  24
CHUVAMM =  10
CHUVAPOR =  90
UMIDMAX =  83
UMIDMIN =  43
VENTOVEL =  6
SOLNASCE =  525
SOLSEPOE =  1746 

ID =  3
TEMPMAX =  38
TEMPMIN =  24
CHUVAMM =  0
CHUVAPOR =  0
UMIDMAX =  77
UMIDMIN =  43
VENTOVEL =  5
SOLNASCE =  525
SOLSEPOE =  1747 

ID =  4
TEMPMAX =  38
TEMPMIN =  24
CHUVAMM =  0
CHUVAPOR =  0
UMIDMAX =  77
UMIDMIN =  43
VENTOVEL =  5
SOLNASCE =  525
SOLSEPOE =  1747 

ID =  5
TEMPMAX =  38
TEMPMIN =  24
CHUVAMM =  0
CHUVAPOR =  0
UMIDMAX =  77
UMIDMIN =  43
VENTOVEL =  5
SOLNASCE =  525
SOLSEPOE =  1747 

ID =  6
TEMPMAX =  38
TEMPMIN =  24
CHUVAMM =  0
CHUVAPOR =  0
UMIDMAX =  77
U


Iniciando conexão com o banco...
Gravação realizada com sucesso!
Finalizando conexão com o banco

Iniciando conexeção com o banco...
ID =  1
TEMPMAX =  36
TEMPMIN =  24
CHUVAMM =  10
CHUVAPOR =  90
UMIDMAX =  83
UMIDMIN =  43
VENTOVEL =  6
SOLNASCE =  -20
SOLSEPOE =  -29 

ID =  2
TEMPMAX =  36
TEMPMIN =  24
CHUVAMM =  10
CHUVAPOR =  90
UMIDMAX =  83
UMIDMIN =  43
VENTOVEL =  6
SOLNASCE =  525
SOLSEPOE =  1746 

ID =  3
TEMPMAX =  38
TEMPMIN =  24
CHUVAMM =  0
CHUVAPOR =  0
UMIDMAX =  77
UMIDMIN =  43
VENTOVEL =  5
SOLNASCE =  525
SOLSEPOE =  1747 

ID =  4
TEMPMAX =  38
TEMPMIN =  24
CHUVAMM =  0
CHUVAPOR =  0
UMIDMAX =  77
UMIDMIN =  43
VENTOVEL =  5
SOLNASCE =  525
SOLSEPOE =  1747 

ID =  5
TEMPMAX =  38
TEMPMIN =  24
CHUVAMM =  0
CHUVAPOR =  0
UMIDMAX =  77
UMIDMIN =  43
VENTOVEL =  5
SOLNASCE =  525
SOLSEPOE =  1747 

ID =  6
TEMPMAX =  38
TEMPMIN =  24
CHUVAMM =  0
CHUVAPOR =  0
UMIDMAX =  77
UMIDMIN =  43
VENTOVEL =  5
SOLNASCE =  525
SOLSEPOE =  1747 

ID =  7
TEMPMAX =  38
T